In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import json
import itertools

In [18]:
'''setup = {
    "db": ["cifar-10"], #["mnist", "cifar-10", "cifar-100", "svhn"]
    "dim": ["d200"], #["d100", "d200", "d500"]
    "do": ["norm", "reg"],
    "on": ["x", "w"],
    "std": ["s1o4", "s1o2", "s1o1"],
    "lyr": ["no", "l1",  "l2", "l3", "all"],
    "run": ["1"],
}'''

setup = {
    "tag": ["none", "base", "mean", "cov", "var"],
    "run": ["1"],
}

def read_metrics(setup):
    dfs = []
    for values in list(itertools.product(*setup.values())):
        label = "_".join(values)
        title = dict(zip(setup.keys(), values))
        try:
            df = pd.read_json(f"outputs/{label}/logs.json")
            df = df.assign(**title)
            df["epoch"] = df.index + 1
            dfs.append(df)
        except:
            pass
    return pd.concat(dfs, ignore_index=True)

def read_checkpoints(setup):
    data = []
    for values in list(itertools.product(*setup.values())):
        label = "_".join(values)
        title = dict(zip(setup.keys(), values))
        try:
            state = torch.load(f"outputs/{label}/checkpoint.pt")
        except:
            continue
        data.append({
            **title,
            **{f"w{i}": w.norm().item() for i, w in enumerate(state["ml"]["w"])},
            **{f"b{i}": b.norm().item() for i, b in enumerate(state["ml"]["b"])},
            **{f"x{i}": m.diagonal().sum().sqrt().item() for i, m in enumerate(state["ml"]["m2"])},
        })

read_checkpoints(setup)

,train_loss,eval_model_2.weight.l1,eval_model_2.weight.l2,eval_model_2.bias.l1,eval_model_2.bias.l2,eval_model_5.weight.l1,eval_model_5.weight.l2,eval_model_5.bias.l1,eval_model_5.bias.l2,eval_model_8.weight.l1,...,eval_model_8.bias.l2,eval_model_11.weight.l1,eval_model_11.weight.l2,eval_model_11.bias.l1,eval_model_11.bias.l2,eval_loss,eval_recall1,tag,run,epoch
0,2.183503,0.009075,0.010514,0.008344,0.010206,0.050375,0.058167,0.045780,0.054516,0.050342,...,0.059800,0.054326,0.062981,0.040626,0.048558,1.993463,0.2844,none,1,1
1,1.881180,0.009166,0.010669,0.009067,0.011281,0.051007,0.058983,0.046819,0.055455,0.050968,...,0.060825,0.060008,0.070406,0.041463,0.050881,1.777954,0.3639,none,1,2
2,1.727290,0.009247,0.010805,0.009921,0.012271,0.051520,0.059667,0.047424,0.056055,0.051497,...,0.061617,0.064234,0.076131,0.038896,0.052805,1.667873,0.4034,none,1,3
3,1.644757,0.009317,0.010924,0.010887,0.013409,0.051942,0.060226,0.047938,0.056602,0.051939,...,0.062509,0.067477,0.080575,0.037249,0.055904,1.607535,0.4286,none,1,4
4,1.583154,0.009386,0.011038,0.011912,0.014785,0.052323,0.060726,0.048422,0.057218,0.052330,...,0.063575,0.070224,0.084386,0.039928,0.060964,1.557674,0.4511,none,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.873335,0.014157,0.017785,0.070681,0.093177,0.058980,0.071842,0.098068,0.134804,0.059707,...,0.124232,0.125564,0.152852,0.247697,0.299232,1.546749,0.5175,var,1,46
246,0.859384,0.014309,0.017981,0.071791,0.094662,0.059160,0.072118,0.099092,0.136257,0.059895,...,0.125157,0.126689,0.154208,0.248507,0.300656,1.552890,0.5199,var,1,47
247,0.852763,0.014461,0.018180,0.072807,0.096082,0.059333,0.072377,0.100128,0.137638,0.060069,...,0.125970,0.127726,0.155458,0.248955,0.301777,1.557548,0.5165,var,1,48
248,0.845348,0.014613,0.018377,0.073821,0.097403,0.059490,0.072612,0.101071,0.138930,0.060223,...,0.126710,0.128618,0.156576,0.249448,0.302861,1.549425,0.5176,var,1,49


In [ ]:
dbs = ["cifar-10"] #["mnist", "cifar-10", "cifar-100", "svhn"]
dims = ["d200"] #["d100", "d200", "d500"]
dos = ["norm", "reg"]
ons = ["x", "w"]
stds = ["s1o4", "s1o2", "s1o1"]
lyrs = ["no", "l1",  "l2", "l3", "all"]

In [ ]:
data = []
for db, dim, do, on, std, lyr in itertools.product(dbs, dims, dos, ons, stds, lyrs):
  label = f"{db}_{dim}_{do}_{on}_{std}_{lyr}_1"
  try:
    state = torch.load(f"outputs/{label}/analysis.pt")
  except:
    continue
  data.append({
    "db": db, "dim": dim, "do": do, "on": on, "std": std, "lyr": lyr,
    **{f"w{i}": w.norm().item() for i, w in enumerate(state["ml"]["w"])},
    **{f"b{i}": b.norm().item() for i, b in enumerate(state["ml"]["b"])},
    **{f"x{i}": m.diagonal().sum().sqrt().item() for i, m in enumerate(state["ml"]["m2"])},
  })
df = pd.DataFrame(data)

mets = {"state": "x", "weight": "w", "bias": "b"}
for i, (db, std) in enumerate(itertools.product(dbs, stds)):
  ncols = len(dos)*len(ons)
  nrows = len(mets)
  _, axes = plt.subplots(
    nrows=nrows, ncols=ncols, sharex="col", sharey="row",
    figsize=(ncols*2.5, nrows*2), dpi=100, facecolor="w")
  for j, (on, do) in enumerate(itertools.product(ons, dos)):
    sdf = df[(df["db"]==db)&(df["dim"]==dim)&(df["do"]==do)&(df["on"]==on)&(df["std"]==std)]
    if len(sdf) == 0:
      continue
    sdf = sdf.set_index("lyr").drop(columns=["db", "dim", "do", "on", "std"])
    sdf = sdf.divide(sdf.loc["no"]).drop("no")
    for k, (met, prefix) in enumerate(mets.items()):
      cols = [col for col in sdf.columns if col.startswith(prefix)]
      sdf[cols].plot.bar(ax=axes[k][j], legend=False, rot=0)
      axes[k][j].set_xlabel(f"{do} {on} {std}")
      axes[k][j].set_ylabel(met)
      axes[k][j].set_ylim((0, 1.5))
      axes[k][j].grid()

In [ ]:
def read(config):
  data = []
  for values in itertools.product(*config.values()):
    label = "_".join(values)+"_1"
    title = {k: v for k, v in zip(config.keys(), values)}
    if title["detach"] == "ans":
      db, dim, do, on, detach, std, lyr = values
      label = f"{db}_{dim}_norm_{on}_{std}_{lyr}_1"
    try:
      with open(f"outputs/{label}/logs.json") as f:
          logs = json.load(f)
      for i, log in enumerate(logs):
          data.append({**title, "epoch": i+1, **log})
    except:
      pass
  return pd.DataFrame(data)

def plot_curve(config, x, y, g):
  df = read(config)
  values = list(itertools.product(*[config[k] for k in x]))
  fields = y

  ncols = len(values)
  nrows = len(fields)
  _, axes = plt.subplots(
    nrows=nrows, ncols=ncols, sharex=True, sharey="row",
    figsize=(ncols*2.5, nrows*2), dpi=100, facecolor="w")

  for i, vs in enumerate(values):
    label = " ".join(vs)
    sdf = df
    for k, v in zip(x, vs):
      sdf = sdf[sdf[k] == v]
    if len(sdf) == 0:
      continue
    sdf = sdf.set_index("epoch").groupby(g)
    for j, field in enumerate(fields):
      ax = axes[j][i]
      sdf[field].plot(ax=ax, legend=False)
      ax.set_xlabel(label)
      ax.set_ylabel(field)
      ax.grid()
      if j == 0:
        ax.legend()
      if field == "eval_top1":
        ax.set_ylim((0.45, 0.6))

config = {
  "db": ["cifar-10"],
  "dim": ["d200"],
  "do": ["reg"],
  "on": ["x", "w"],
  "detach": ["ans", "no"],
  "std": ["s1o1"],
  "lyr": ["l1", "l2", "l3"],
}

metsets = [
  ["train_loss", "eval_loss", "eval_top1"], 
  [f"train_model_{i}.state.l1" for i in (4, 6, 8)] + ["train_model.output.l1"],
  [f"eval_model_{i}.weight.l1" for i in (1, 3, 5, 7)],
]
for mets in metsets:
  plot_curve(config, x=["do", "std", "on", "std", "lyr"], y=mets, g="detach")


In [ ]:
data = []
for db, dim, do, on, std, lyr in itertools.product(dbs, dims, dos, ons, stds, lyrs):
  label = f"{db}_{dim}_{do}_{on}_{std}_{lyr}_1"
  title = {"db": db, "dim": dim, "do": do, "std": std, "lyr": lyr, "on": on}
  try:
    with open(f"outputs/{label}/logs.json") as f:
        logs = json.load(f)
    for i, log in enumerate(logs):
        data.append({**title, "epoch": i+1, **log})
  except:
    pass
df = pd.DataFrame(data)

metsets = [
  ["train_loss", "eval_loss", "eval_top1"], 
  [f"train_model_{i}.state.l1" for i in (4, 6, 8)] + ["train_model.output.l1"],
  #[f"eval_model_{i}.weight.l1" for i in (1, 3, 5, 7)],
]
for i, (db, on, mets) in enumerate(itertools.product(dbs, ons[:1], metsets)):
  ncols = len(dos)*len(stds)
  nrows = len(mets)
  _, axes = plt.subplots(
    nrows=nrows, ncols=ncols, sharex=True, sharey="row",
    figsize=(ncols*2.5, nrows*2), dpi=100, facecolor="w")
  for j, (do, std) in enumerate(itertools.product(dos, stds)):
    sdf = df[(df["db"]==db)&(df["dim"]==dim)&(df["do"]==do)&(df["std"]==std)&(df["on"]==on)]
    if len(sdf) == 0:
      continue
    sdf = sdf.drop(columns=["db", "dim", "do", "on", "std"])
    sdf = sdf.set_index("epoch").groupby("lyr")
    for k, met in enumerate(mets):
      sdf[met].plot(ax=axes[k][j], legend=False)
      axes[k][j].set_xlabel(f"{do} {on} {std}")
      axes[k][j].set_ylabel(met)
      axes[k][j].grid()
      if met == "eval_top1":
        axes[k][j].set_ylim((0.45, 0.6))
    axes[0][j].legend()

In [ ]:
def plot(db, dim, do, on, std, lyr, layers, flip, axes):
  ylim, xlim = [0, 2], [-.25, .25]
  label = f"{db}_{dim}_{do}_{on}_{std}_{lyr}_1"
  try:
    state = torch.load(f"outputs/{label}/analysis.pt")
  except:
    return
  for j, layer in enumerate(layers):
    w = state["ml"]["w"][layer] if flip else state["ml"]["w"][layer+1].T
    x = torch.matmul(w,  w.T)
    y = state["ml"]["m2"][layer]
    axes[-j-1].hist2d(
      x.flatten().numpy(), y.flatten().numpy(), 
      range=[xlim, ylim], bins=[50, 50],
      norm=mpl.colors.LogNorm())
  axes[-1].set_xlabel(f"{do} {on} {std} {lyr}")

layers = range(3)
for db, dim, on, lyr in itertools.product(dbs, dims, ons, lyrs[1:]):
  nrows = len(layers)
  ncols = len(dos)*len(stds)
  _, axes = plt.subplots(
    nrows=nrows, ncols=ncols, sharex=True, sharey=True,
    figsize=(ncols*2, nrows*2), dpi=100, facecolor="w")
  for i, (do, std) in enumerate(itertools.product(dos, stds)):
    sect = [row[i] for row in axes]
    plot(db, dim, do, on, std, lyr, layers, False, sect)